In [35]:
from datascience.utils.common import read_yaml, create_directories, save_json
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
from datascience.constants import *
from dataclasses import dataclass
from pathlib import Path

import mlflow.sklearn
import pandas as pd
import numpy as np
import joblib
import os

In [36]:
# os.chdir("../")   for write in base_dir
%pwd

'C:\\Users\\Darius\\PycharmProjects\\Mlops_Project_1'

In [37]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Bahna-Darius/Mlops_Project_1.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Bahna-Darius"
os.environ["MLFLOW_TRACKING_PASSWORD"]="4f1f95f843b7c12494324c9ee8db37fddb02a368"
os.environ["MLFLOW_TRACKING_SERVER_SUPPORTS_LOGGED_MODEL"] = "false"

In [38]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [39]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Bahna-Darius/Mlops_Project_1.mlflow"
        )

        return model_evaluation_config


In [40]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model")

            else:
                mlflow.sklearn.log_model(model, "model")



In [41]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-10-23 18:04:20,737-> INFO-> common-> yaml file: C:\Users\Darius\PycharmProjects\Mlops_Project_1\src\datascience\config\config.yaml loaded successfully]
[2025-10-23 18:04:20,737-> INFO-> common-> yaml file: C:\Users\Darius\PycharmProjects\Mlops_Project_1\params.yaml loaded successfully]
[2025-10-23 18:04:20,737-> INFO-> common-> yaml file: C:\Users\Darius\PycharmProjects\Mlops_Project_1\schema.yaml loaded successfully]
[2025-10-23 18:04:20,747-> INFO-> common-> created directory at: artifacts]
[2025-10-23 18:04:20,747-> INFO-> common-> created directory at: artifacts/model_evaluation]
[2025-10-23 18:04:21,423-> INFO-> common-> json file saved at: artifacts\model_evaluation\metrics.json]


2025/10/23 18:04:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run tasteful-flea-572 at: https://dagshub.com/Bahna-Darius/Mlops_Project_1.mlflow/#/experiments/0/runs/3b67d4e6aa0c49eaa0c86996f912d20a
🧪 View experiment at: https://dagshub.com/Bahna-Darius/Mlops_Project_1.mlflow/#/experiments/0


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}